# Evaluating Coccinelle Semantic Patches in linux-mainline

### Setting up the notebook

In [1]:
# Import all needed libraries
import json
import calendar
import random
import os
import sys
from datetime import date, timedelta

import faker
import numpy as np
import pandas as pd
from pandas import DataFrame, read_csv
from delorean import parse
import matplotlib

# Enable inline plotting
%matplotlib inline

In [2]:
print('Python version ' + sys.version)
print('Pandas version ' + pd.__version__)
print('Matplotlib version ' + matplotlib.__version__)

Python version 3.6.3 (v3.6.3:2c5fed8, Oct  3 2017, 18:11:49) [MSC v.1900 64 bit (AMD64)]
Pandas version 0.20.3
Matplotlib version 2.1.0


### Preparing the data

In [3]:
# Dictionary listing all Semantic Patches in linux-4.14-rc6
# TODO: Should refactor into file 'cocci_linux_4_14_rc6.json'

cocci_linux_4_14_rc6 = {
    'info': {
        'repository': 'linux-4.14-rc6'
    },
    'scripts': [
        {
            'path': 'scripts/coccinelle/api/alloc',
            'filename': 'alloc_cast.cocci',
            'confidence': 'High',
            'description': "Remove casting the values returned by memory allocation functions like "
                "kmalloc, kzalloc, kmem_cache_alloc, kmem_cache_zalloc etc. "
                "This makes an effort to find cases of casting of values returned "
                "by kmalloc, kzalloc, kcalloc, kmem_cache_alloc, kmem_cache_zalloc, "
                "kmem_cache_alloc_node, kmalloc_node and kzalloc_node and removes the casting "
                "as it is not required. The result in the patch case may need some reformatting."
        },
        {
            'path': 'scripts/coccinelle/api/alloc',
            'filename': 'kzalloc-simple.cocci',
            'confidence': 'High',
            'description': """use kzalloc rather than..."""
        },
        {
            'path': 'scripts/coccinelle/api/alloc',
            'filename': 'pool_zalloc-simple.cocci',
            'confidence': 'N.A.',
            'description': """Use *_pool_zalloc rather than..."""
        },
        # TODO: Complete with all the 59 entries
        {
            'path': 'scripts/coccinelle/api',
            'filename': 'd_find_alias.cocci',
            'confidence': 'Moderate',
            'description': "?"
        },
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'debugfs_simple_attr.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'drm-get-pub.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'err_cast.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'kstrdup.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'memdup.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'memdup_user.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'platform_no_drv_owner.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'pm_runtime.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'ptr_ret.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'resource_size.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'setup_timer.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'simple_open.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'vma_pages.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'clk_put.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'devm_free.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'ifnulfree.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'iounmap.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'kfree.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'kfreeaddr.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'pci_free_consistent.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'device_node_continue.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'fen.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'ifnull.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'list_entry_update.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'use_after_iter.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'call_kern.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'double_lock.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'flags.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'mini_lock.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'array_size.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'badty.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'boolconv.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'boolinit.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'boolreturn.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'bugon.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'cond_no_effect.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'cstptr.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'doubleinit.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'ifaddr.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'ifcol.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'irqf_oneshot.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'noderef.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'of_table.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'orplus.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'returnvar.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'semicolon.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'warn.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'badzero.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'deref_null.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'eno.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'kmerr.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'doublebitand.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'doubletest.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'odd_ptr_err.cocci',},
        {
            'path': 'scripts/coccinelle/?',
            'filename': 'unsigned_lesser_than_zero.cocci',},
    ]
}

# cocci_linux_4_14_rc6

In [4]:
print(cocci_linux_4_14_rc6)
len(cocci_linux_4_14_rc6['scripts'])

{'info': {'repository': 'linux-4.14-rc6'}, 'scripts': [{'path': 'scripts/coccinelle/api/alloc', 'filename': 'alloc_cast.cocci', 'confidence': 'High', 'description': 'Remove casting the values returned by memory allocation functions like kmalloc, kzalloc, kmem_cache_alloc, kmem_cache_zalloc etc. This makes an effort to find cases of casting of values returned by kmalloc, kzalloc, kcalloc, kmem_cache_alloc, kmem_cache_zalloc, kmem_cache_alloc_node, kmalloc_node and kzalloc_node and removes the casting as it is not required. The result in the patch case may need some reformatting.'}, {'path': 'scripts/coccinelle/api/alloc', 'filename': 'kzalloc-simple.cocci', 'confidence': 'High', 'description': 'use kzalloc rather than...'}, {'path': 'scripts/coccinelle/api/alloc', 'filename': 'pool_zalloc-simple.cocci', 'confidence': 'N.A.', 'description': 'Use *_pool_zalloc rather than...'}, {'path': 'scripts/coccinelle/api', 'filename': 'd_find_alias.cocci', 'confidence': 'Moderate', 'description': '?

59

### Creating the DataFrame for cocci_linux_4_14_rc6

In [5]:
data = cocci_linux_4_14_rc6['scripts']
df = DataFrame(data) # , index=cocci_linux_4_14_rc6['scripts']['filename']

#df.count()
#df.describe()
#json.dumps(data)
#print(df.index)
#print()
#print(df.filename)
#print()

#df.head()

In [6]:
# Rearrange columns
cols = ['path', 'filename', 'confidence', 'description']
df = df[cols]

df

,path,filename,confidence,description
0,scripts/coccinelle/api/alloc,alloc_cast.cocci,High,Remove casting the values returned by memory a...
1,scripts/coccinelle/api/alloc,kzalloc-simple.cocci,High,use kzalloc rather than...
2,scripts/coccinelle/api/alloc,pool_zalloc-simple.cocci,N.A.,Use *_pool_zalloc rather than...
3,scripts/coccinelle/api,d_find_alias.cocci,Moderate,?
4,scripts/coccinelle/?,debugfs_simple_attr.cocci,NaN,NaN
5,scripts/coccinelle/?,drm-get-pub.cocci,NaN,NaN
6,scripts/coccinelle/?,err_cast.cocci,NaN,NaN
7,scripts/coccinelle/?,kstrdup.cocci,NaN,NaN
8,scripts/coccinelle/?,memdup.cocci,NaN,NaN
9,scripts/coccinelle/?,memdup_user.cocci,NaN,NaN


In [7]:
# Notice that text of column "description" as shown above is truncated,
# however the cell inside the DataFrame contains the full value
df.iloc[0]['description']

'Remove casting the values returned by memory allocation functions like kmalloc, kzalloc, kmem_cache_alloc, kmem_cache_zalloc etc. This makes an effort to find cases of casting of values returned by kmalloc, kzalloc, kcalloc, kmem_cache_alloc, kmem_cache_zalloc, kmem_cache_alloc_node, kmalloc_node and kzalloc_node and removes the casting as it is not required. The result in the patch case may need some reformatting.'

In [8]:
# Display full text for column "description"
# See <https://stackoverflow.com/questions/23388810/ipython-notebook-output-cell-is-truncating-contents-of-my-list>
# See <http://pandas.pydata.org/pandas-docs/stable/options.html>

from pandas import DataFrame
from IPython.display import HTML

pd.options.display.max_colwidth = 2000
pd.options.display.max_seq_items = 200

HTML(df.to_html())

,path,filename,confidence,description
0,scripts/coccinelle/api/alloc,alloc_cast.cocci,High,"Remove casting the values returned by memory allocation functions like kmalloc, kzalloc, kmem_cache_alloc, kmem_cache_zalloc etc. This makes an effort to find cases of casting of values returned by kmalloc, kzalloc, kcalloc, kmem_cache_alloc, kmem_cache_zalloc, kmem_cache_alloc_node, kmalloc_node and kzalloc_node and removes the casting as it is not required. The result in the patch case may need some reformatting."
1,scripts/coccinelle/api/alloc,kzalloc-simple.cocci,High,use kzalloc rather than...
2,scripts/coccinelle/api/alloc,pool_zalloc-simple.cocci,N.A.,Use *_pool_zalloc rather than...
3,scripts/coccinelle/api,d_find_alias.cocci,Moderate,?
4,scripts/coccinelle/?,debugfs_simple_attr.cocci,NaN,NaN
5,scripts/coccinelle/?,drm-get-pub.cocci,NaN,NaN
6,scripts/coccinelle/?,err_cast.cocci,NaN,NaN
7,scripts/coccinelle/?,kstrdup.cocci,NaN,NaN
8,scripts/coccinelle/?,memdup.cocci,NaN,NaN
9,scripts/coccinelle/?,memdup_user.cocci,NaN,NaN


In [9]:
# Export the dataframe to an Excel file
#df.to_excel('test.xls')

### Creating DataFrames for instrument-coccicheck reports 

In [10]:
# Report against linux-4.14-rc6 full-tree

report_linux_4_14_rc6_fulltree = {
    'info': {
        'note': """Report against linux-4.14-rc6 full-tree""",
        'script_start': 'Thu Oct  5 13:11:29 CEST 2017',
        'src_sha': 'd81fa669e3de7eb8a631d7d95dac5fbcb2bf9d4e',
    },
    'statistics': [
        {'filename': 'alloc_cast.cocci',         'L':  13, 'W':  0, 'E': 0},
        {'filename': 'kzalloc-simple.cocci',     'L':   4, 'W':  4, 'E': 0},
        {'filename': 'pool_zalloc-simple.cocci', 'L':  22, 'W': 22, 'E': 0}, 
        # TODO: Complete with all the 59 entries
    ]
}

In [11]:
data2_full = DataFrame(report_linux_4_14_rc6_fulltree['statistics'])
data2_full = data2_full[['filename','L', 'W', 'E']]

data2_full

,filename,L,W,E
0,alloc_cast.cocci,13,0,0
1,kzalloc-simple.cocci,4,4,0
2,pool_zalloc-simple.cocci,22,22,0


In [12]:
#data2_full.plot()
#data2_full[['L', 'W', 'E']].plot()
#data2_full[['filename','L', 'W', 'E']].plot(kind='bar')

#len(data2_full['filename'].unique())

In [13]:
# Report against linux-4.4.50 minimized-tree

report_linux_4_4_50_minimizedtree = {
    'info': {
        'note': """Report against linux-4.4.50 minimzed-tree""",
        'script_start': 'Mon Oct 23 16:29:52 IST 2017',
        'cocci_sha': '9e66317d3c92ddaab330c125dfe9d06eee268aff',
        'src_sha': '90dcab23bbc81fbfa25dfdb91d4ce974a69bd210',
    },
    'statistics': [
        {'filename': 'alloc_cast.cocci',         'L':  2, 'W': 0, 'E': 0},
        {'filename': 'kzalloc-simple.cocci',     'L':  0, 'W': 0, 'E': 0},
        {'filename': 'pool_zalloc-simple.cocci', 'L':  5, 'W': 0, 'E': 0}, 
        # TODO: Complete with all the 59 entries
    ]
}

In [14]:
data3_minimized = DataFrame(report_linux_4_4_50_minimizedtree['statistics'])
data3_minimized = data3_minimized[['filename','L', 'W', 'E']]

data3_minimized

,filename,L,W,E
0,alloc_cast.cocci,2,0,0
1,kzalloc-simple.cocci,0,0,0
2,pool_zalloc-simple.cocci,5,0,0


### Assigning estimated safety relevance to cocci scripts

In [15]:
my_safetyrel_cocci = {
    'info': {
        # TODO
    },
    # rel: Safety Relevance (1=Highest, 9=lowest)
    'values': [
        {'filename': 'alloc_cast.cocci',                'rel': 5, 'notes': ''},
        {'filename': 'kzalloc-simple.cocci',            'rel': 5, 'notes': ''},
        {'filename': 'pool_zalloc-simple.cocci',        'rel': 8, 'notes': ''},
        {'filename': 'd_find_alias.cocci',              'rel': 3, 'notes': ''},
        {'filename': 'debugfs_simple_attr.cocci',       'rel': 6, 'notes': ''},
        {'filename': 'drm-get-pub.cocci',               'rel': 6, 'notes': ''},
        {'filename': 'err_cast.cocci',                  'rel': 5, 'notes': ''},
        {'filename': 'kstrdup.cocci',                   'rel': 4, 'notes': ''},
        {'filename': 'memdup.cocci',                    'rel': 4, 'notes': ''},
        {'filename': 'memdup_user.cocci',               'rel': 4, 'notes': ''},
        {'filename': 'platform_no_drv_owner.cocci',     'rel': 5, 'notes': ''},
        {'filename': 'pm_runtime.cocci',                'rel': 4, 'notes': ''},
        {'filename': 'ptr_ret.cocci',                   'rel': 5, 'notes': ''},
        {'filename': 'resource_size.cocci',             'rel': 4, 'notes': ''},
        {'filename': 'setup_timer.cocci',               'rel': 6, 'notes': ''},
        {'filename': 'simple_open.cocci',               'rel': 4, 'notes': ''},
        {'filename': 'vma_pages.cocci',                 'rel': 4, 'notes': ''},
        {'filename': 'clk_put.cocci',                   'rel': 4, 'notes': ''},
        {'filename': 'devm_free.cocci',                 'rel': 2, 'notes': 'Beware of false positives'},
        {'filename': 'ifnulfree.cocci',                 'rel': 7, 'notes': ''},
        {'filename': 'iounmap.cocci',                   'rel': 4, 'notes': 'Beware of false positives'},
        {'filename': 'kfree.cocci',                     'rel': 2, 'notes': 'Beware of false positives'},
        {'filename': 'kfreeaddr.cocci',                 'rel': 1, 'notes': ''},
        {'filename': 'pci_free_consistent.cocci',       'rel': 4, 'notes': ''},
        {'filename': 'device_node_continue.cocci',      'rel': 3, 'notes': ''},
        {'filename': 'fen.cocci',                       'rel': 5, 'notes': ''},
        {'filename': 'ifnull.cocci',                    'rel': 5, 'notes': 'Beware of false positives'},
        {'filename': 'list_entry_update.cocci',         'rel': 4, 'notes': ''},
        {'filename': 'use_after_iter.cocci',            'rel': 3, 'notes': 'Beware of false positives'},
        {'filename': 'call_kern.cocci',                 'rel': 4, 'notes': ''},
        {'filename': 'double_lock.cocci',               'rel': 5, 'notes': 'Beware of false positives'},
        {'filename': 'flags.cocci',                     'rel': 2, 'notes': ''},
        {'filename': 'mini_lock.cocci',                 'rel': 5, 'notes': 'Beware of false positives'},
        {'filename': 'array_size.cocci',                'rel': 4, 'notes': ''},
        {'filename': 'badty.cocci',                     'rel': 4, 'notes': 'Beware of false positives'},
        {'filename': 'boolconv.cocci',                  'rel': 9, 'notes': 'Cleanup code only'},
        {'filename': 'boolinit.cocci',                  'rel': 8, 'notes': ''},
        {'filename': 'boolreturn.cocci',                'rel': 6, 'notes': 'Does not affect code behaviour'},
        {'filename': 'bugon.cocci',                     'rel': 8, 'notes': ''},
        {'filename': 'cond_no_effect.cocci',            'rel': 4, 'notes': 'Developed by Nicholas Mc Guire, OSADL'},
        {'filename': 'cstptr.cocci',                    'rel': 1, 'notes': ''},
        {'filename': 'doubleinit.cocci',                'rel': 7, 'notes': 'High rate of false positives'},
        {'filename': 'ifaddr.cocci',                    'rel': 1, 'notes': ''},
        {'filename': 'ifcol.cocci',                     'rel': 7, 'notes': 'High rate of false positives'},
        {'filename': 'irqf_oneshot.cocci',              'rel': 6, 'notes': ''},
        {'filename': 'noderef.cocci',                   'rel': 2, 'notes': ''},
        {'filename': 'of_table.cocci',                  'rel': 2, 'notes': ''},
        {'filename': 'orplus.cocci',                    'rel': 4, 'notes': 'Beware of false positives'},
        {'filename': 'returnvar.cocci',                 'rel': 7, 'notes': ''},
        {'filename': 'semicolon.cocci',                 'rel': 8, 'notes': ''},
        {'filename': 'warn.cocci',                      'rel': 4, 'notes': ''},
        {'filename': 'badzero.cocci',                   'rel': 5, 'notes': ''},
        {'filename': 'deref_null.cocci',                'rel': 4, 'notes': 'Beware of false positives'},
        {'filename': 'eno.cocci',                       'rel': 2, 'notes': ''},
        {'filename': 'kmerr.cocci',                     'rel': 2, 'notes': ''},
        {'filename': 'doublebitand.cocci',              'rel': 7, 'notes': 'Beware of false positives'},
        {'filename': 'doubletest.cocci',                'rel': 7, 'notes': 'Beware of false positives'},
        {'filename': 'odd_ptr_err.cocci',               'rel': 4, 'notes': ''},
        {'filename': 'unsigned_lesser_than_zero.cocci', 'rel': 3, 'notes': ''},
        # TODO: Complete with all the 59 entries
    ]
}

In [16]:
data5_relevance = DataFrame(my_safetyrel_cocci['values'])
data5_relevance = data5_relevance[['filename','rel', 'notes']]

data5_relevance

,filename,rel,notes
0,alloc_cast.cocci,5,
1,kzalloc-simple.cocci,5,
2,pool_zalloc-simple.cocci,8,
3,d_find_alias.cocci,3,
4,debugfs_simple_attr.cocci,6,
5,drm-get-pub.cocci,6,
6,err_cast.cocci,5,
7,kstrdup.cocci,4,
8,memdup.cocci,4,
9,memdup_user.cocci,4,


### Merge tables and aggregate results

In [17]:
#results = data5_relevance.copy(deep=True)
#results.sort_values(by='rel')

#print('DEBUG: id(data5_relevance):', id(data5_relevance))
#print('DEBUG: id(results):', id(results))
#print('DEBUG: len(results):', len(results))
#print('')

#for fn in results['filename']:
#    print('DEBUG: fn=', fn)

#print("DEBUG: df=", df)
#print("DEBUG: df.index=", df.index)
#print("DEBUG: df['filename']=", df['filename'])
#print('')

#results2 = DataFrame(index=results['filename'])
#print("DEBUG: results2=", results2)
#print("DEBUG: results2.index=", results2.index)

#print(results2)

#for ix in results.index:
#    #print('DEBUG: ix=', ix)
#    fn = results['filename'].get(ix)
#    rel = results['rel'].get(ix)
#    notes = results['notes'].get(ix)
#    print('DEBUG: ix=', ix, ', fn=', fn, ', rel=', rel, ', notes=', notes)
#    #results2.append({
#    #    'filename': fn,
#    #    'rel': rel,
#    #    'notes': notes,
#    #}, ignore_index=True)
#    
#    results2[fn]['rel'] = rel
#    results2[fn]['notes'] = notes
#    # TODO

In [18]:
data5_relevance

,filename,rel,notes
0,alloc_cast.cocci,5,
1,kzalloc-simple.cocci,5,
2,pool_zalloc-simple.cocci,8,
3,d_find_alias.cocci,3,
4,debugfs_simple_attr.cocci,6,
5,drm-get-pub.cocci,6,
6,err_cast.cocci,5,
7,kstrdup.cocci,4,
8,memdup.cocci,4,
9,memdup_user.cocci,4,


In [19]:
df

,path,filename,confidence,description
0,scripts/coccinelle/api/alloc,alloc_cast.cocci,High,"Remove casting the values returned by memory allocation functions like kmalloc, kzalloc, kmem_cache_alloc, kmem_cache_zalloc etc. This makes an effort to find cases of casting of values returned by kmalloc, kzalloc, kcalloc, kmem_cache_alloc, kmem_cache_zalloc, kmem_cache_alloc_node, kmalloc_node and kzalloc_node and removes the casting as it is not required. The result in the patch case may need some reformatting."
1,scripts/coccinelle/api/alloc,kzalloc-simple.cocci,High,use kzalloc rather than...
2,scripts/coccinelle/api/alloc,pool_zalloc-simple.cocci,N.A.,Use *_pool_zalloc rather than...
3,scripts/coccinelle/api,d_find_alias.cocci,Moderate,?
4,scripts/coccinelle/?,debugfs_simple_attr.cocci,NaN,NaN
5,scripts/coccinelle/?,drm-get-pub.cocci,NaN,NaN
6,scripts/coccinelle/?,err_cast.cocci,NaN,NaN
7,scripts/coccinelle/?,kstrdup.cocci,NaN,NaN
8,scripts/coccinelle/?,memdup.cocci,NaN,NaN
9,scripts/coccinelle/?,memdup_user.cocci,NaN,NaN


In [20]:
# TODO: Join df on filename
# result['path'] = pd.Series()
# TODO: Join data2_full on filename
# TODO: Join data3_minimize on filename

#results2

# Tutorial: Merge and Join dataframes with Panda
# <https://www.shanelynn.ie/merge-join-dataframes-python-pandas-index-1/>

result = pd.merge(data5_relevance,
                  df,
                  on='filename')

result.sort_values(by='rel')

,filename,rel,notes,path,confidence,description
22,kfreeaddr.cocci,1,,scripts/coccinelle/?,NaN,NaN
42,ifaddr.cocci,1,,scripts/coccinelle/?,NaN,NaN
40,cstptr.cocci,1,,scripts/coccinelle/?,NaN,NaN
54,kmerr.cocci,2,,scripts/coccinelle/?,NaN,NaN
53,eno.cocci,2,,scripts/coccinelle/?,NaN,NaN
18,devm_free.cocci,2,Beware of false positives,scripts/coccinelle/?,NaN,NaN
31,flags.cocci,2,,scripts/coccinelle/?,NaN,NaN
46,of_table.cocci,2,,scripts/coccinelle/?,NaN,NaN
45,noderef.cocci,2,,scripts/coccinelle/?,NaN,NaN
21,kfree.cocci,2,Beware of false positives,scripts/coccinelle/?,NaN,NaN
